### ĐỒ ÁN CUỐI KỲ


In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import sklearn 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import set_config
set_config(display='diagram')
import sklearn
sklearn.__version__

'0.23.1'

## Khám phá dữ liệu (đủ để có thể xác định câu hỏi)

In [2]:
hotels_df = pd.read_csv('data_hotels.csv')
hotels_df.head()

,Name,District,Star,RatingScore,Num_ratings,Price(VND),Pool,Parking,Restaurant
0,Angel Hotel Danang,Sơn Trà,3.0,8.6,287,162790,No,Yes,Yes
1,Stella Maris Beach Danang,Sơn Trà,4.0,8.9,63,832500,No,Yes,Yes
2,Khách sạn DLG Đà Nẵng,Sơn Trà,4.5,9.3,62,1026908,No,Yes,Yes
3,San Marino Boutique Danang,Sơn Trà,4.0,9.0,212,600000,No,Yes,Yes
4,Khách sạn Raon Danang Beach,Ngũ Hành Sơn,3.0,8.8,613,337920,No,No,Yes


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [3]:
hotels_df.shape

(489, 9)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

### Dữ liệu có các dòng bị lặp không?

In [4]:
hotels_df.index.duplicated().sum()

0

### Mỗi cột có ý nghĩa gì?
Mô tả trong file "description.txt".

In [5]:
with open('description.txt', 'r',encoding='utf-8') as f:
    print(f.read())

DỮ LIỆU THÔNG TIN CƠ BẢN CỦA KHÁCH SẢN 

VARIABLE DESCRIPTIONS:

Name			Tên khách sạn

District		Quận; huyện của khách sạn
	
Star			Số sao của khách sạn

RatingScore		Điểm đánh giá khách sạn
	
Num_ratings		Số người đánh giá khách sạn

Price(VND)		Giá thuê phòng (1 đêm/2 người) của khách sạn
	
Pool			Hồ bơi 
			(Yes; No)
Parking			Chỗ đậu xe
			(Yes; No)
Restaurant		Nhà hàng
			(Yes; No)





## Đưa ra câu hỏi cần trả lời

*Output - giá phòng khách sạn -* được tính từ *input - các thông tin của khách sạn-* theo công thức nào?

# Ý nghĩa:
* Khi khách hàng muốn tìm khách sạn với mức giá hợp lý và chất lượng đảm bảo.
* Khi muốn kinh doanh hoặc đầu tư vào một khách sạn.

## Khám phá dữ liệu (để biết cách tách các tập)

In [6]:
# Cột output hiện có kiểu dữ liệu gì?
hotels_df['Price(VND)'].dtype

dtype('int64')

In [7]:
# Cột output có giá trị thiếu không?
hotels_df['Price(VND)'].isna().sum()

0

Do cột output có dữ liệu "liên tục":
* Rời rạc hóa thủ công
* Ép kiểu string

In [8]:
hotels_df['Price(VND)']=pd.cut(hotels_df['Price(VND)'], 10)
hotels_df['Price(VND)']=hotels_df['Price(VND)'].astype(str)
hotels_df.dtypes

Name            object
District        object
Star           float64
RatingScore    float64
Num_ratings      int64
Price(VND)      object
Pool            object
Parking         object
Restaurant      object
dtype: object

Lấy "trần" cột RatingScore
Đổi kiểu dữ liệu int --> float

In [9]:
def ceil_(s):
    return np.ceil(s)
hotels_df['RatingScore']=hotels_df['RatingScore'].apply(ceil_)
hotels_df['Num_ratings'] = hotels_df['Num_ratings'].astype(float)
hotels_df.head()

,Name,District,Star,RatingScore,Num_ratings,Price(VND),Pool,Parking,Restaurant
0,Angel Hotel Danang,Sơn Trà,3.0,9.0,287.0,"(127339.018, 1158850.2]",No,Yes,Yes
1,Stella Maris Beach Danang,Sơn Trà,4.0,9.0,63.0,"(127339.018, 1158850.2]",No,Yes,Yes
2,Khách sạn DLG Đà Nẵng,Sơn Trà,4.5,10.0,62.0,"(127339.018, 1158850.2]",No,Yes,Yes
3,San Marino Boutique Danang,Sơn Trà,4.0,9.0,212.0,"(127339.018, 1158850.2]",No,Yes,Yes
4,Khách sạn Raon Danang Beach,Ngũ Hành Sơn,3.0,9.0,613.0,"(127339.018, 1158850.2]",No,No,Yes


## Tiền xử lý (tách các tập)

In [10]:
# Tách X và y
y = hotels_df["Price(VND)"] 
X = hotels_df.drop("Price(VND)", axis=1)

# Tách tập huấn luyện và tập train theo tỉ lệ 80%:20%
X_train1, X_test, y_train1, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    
# Tách tập huấn luyện và tập validation theo tỉ lệ 80%:20%
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train1, test_size=0.2, random_state=1)


In [11]:
X_train.shape

(312, 8)

In [12]:
y_train.shape

(312,)

In [13]:
X_val.shape

(79, 8)

In [14]:
y_val.shape

(79,)

In [15]:
X_test.shape

(98, 8)

In [16]:
y_test.shape

(98,)

## Khám phá dữ liệu (tập huấn luyện)

### Mỗi cột input hiện đang có kiểu dữ liệu gì?

In [17]:
X_train.dtypes

Name            object
District        object
Star           float64
RatingScore    float64
Num_ratings    float64
Pool            object
Parking         object
Restaurant      object
dtype: object

### Với mỗi cột input có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

Chỉ có cột "Num_ratings" là dạng số thật sự.
Còn Star và RatingScore là dạn cate

In [18]:
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
nume_cols = ['Star', 'RatingScore', 'Num_ratings']
df = X_train[nume_cols]
df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,Star,RatingScore,Num_ratings
missing_ratio,0.0,0.0,0.0
min,0.0,0.0,0.0
lower_quartile,2.0,0.0,0.0
median,2.5,9.0,19.0
upper_quartile,3.0,9.0,68.0
max,5.0,10.0,1032.0


### Với mỗi cột input có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

In [19]:
pd.set_option('display.max_colwidth', 200)
cate_cols = list(set(X_train.columns) - set(nume_cols))
df = X_train[cate_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df.agg([missing_ratio, num_values, value_ratios])

,District,Restaurant,Name,Pool,Parking
missing_ratio,0,0,0,0,0
num_values,7,2,312,2,2
value_ratios,"{'Sơn Trà': 42.3, 'Ngũ Hành Sơn': 28.5, 'Quận Hải Châu': 21.5, 'Thanh Khê': 4.2, 'Đà Nẵng': 1.9, 'Quận Cẩm Lệ': 1.0, 'Liên Chiểu': 0.6}","{'Yes': 65.1, 'No': 34.9}","{'Sơn Trà Green': 0.3, 'Khách sạn Bé Anh': 0.3, 'Hadiva Boutique Hotel': 0.3, 'Hung Anh Hotel': 0.3, 'Khách sạn Legacy Đà Nẵng': 0.3, 'Kangaroo Holiday Lodge 1': 0.3, 'Khách sạn Ngọc Lan': 0.3, 'T...","{'No': 84.3, 'Yes': 15.7}","{'Yes': 53.2, 'No': 46.8}"


## Tiền xử lý (tập huấn luyện) 

* Bỏ cột "Name"
* Chuyển Yes/No --> 1/0

In [20]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self,X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        dropped_cols=["Name"]
        df = X_df.copy()
        df.drop(dropped_cols, axis=1, inplace=True)
        df= df.replace('Yes', 1)
        df= df.replace('No', 0)
        return df

## Tiền xử lý

* Với cột dạng số "Num_rating": điền giá trị thiếu với median
* Với cột dạng số 'Star','RatingScore','Pool','Parking','Restaurant': điền giá trị thiếu với most_frequent
* Với cột 'District': điền giá trị thiếu với most_frequent, sau đó chuyển sang dạng số bằng OneHotEncoder

In [21]:

nume_cols=['Num_ratings']
cate1_cols=['District']
cate2_cols=['Star','RatingScore','Pool','Parking','Restaurant']


col_adderdropper = ColAdderDropper()
imp_median = SimpleImputer(strategy='median') 
imp_mode = SimpleImputer(strategy='most_frequent')

encoding = OneHotEncoder(handle_unknown='ignore', sparse=False)

categorical_transformer = make_pipeline(imp_mode, encoding)

colTransform = ColumnTransformer(transformers=[('numerical', imp_median, nume_cols),\
                                               ('cate2', imp_mode, cate2_cols),
                                               ('cate1', categorical_transformer, cate1_cols)])

colNormalize = StandardScaler()
preprocess_pipeline



## Mô hình hóa với LogisticRegression

In [22]:
log_reg_model = LogisticRegression()
full_pipeline = make_pipeline(col_adderdropper, colTransform, colNormalize, log_reg_model)
full_pipeline

Pipeline(steps=[('coladderdropper', ColAdderDropper()),
                ('columntransformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  SimpleImputer(strategy='median'),
                                                  ['Num_ratings']),
                                                 ('cate2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Star', 'RatingScore',
                                                   'Pool', 'Parking',
                                                   'Restaurant']),
                                                 ('cate1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['District'])])),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

Kiểm tra độ chính xác của mô hình

In [23]:
full_pipeline.fit(X_train, y_train);
train_score = full_pipeline.score(X_train, y_train)
val_score = full_pipeline.score(X_val, y_val)

print(train_score,val_score)

0.8814102564102564 0.8481012658227848


In [24]:
full_pipeline.fit(X_train1, y_train1);
full_pipeline.score(X_test, y_test)

0.8367346938775511

### Nhận xét:
Ưu điểm: độ chính xác cao ~84%
Nhược điểm: Dữ liệu đầu vào nhỏ